# Evaluating SETI detection performance - parameter sweeps over multiple raw files
This notebook runs multiple instances of notebook "02_seti_param_sweep.ipynb", which does the following over multiple RAW files:
1) Inputs each RAW file 
2) Runs Rawspec to generate a filterbank .h5 spectrogram file with specified fine FFT size and integration factor n_sti
3) Runs TurboSETI and/or seticore and compiles a list of detections and compute time (wall clock)
4) Repeats the end-to-end runs over a range of specified fine FFT size and integration factor n_sti

This notebook sequences through multiple RAW files.  Parameters are swept and plots are created in the called notebooks.

Note that these sweeps can be done with various branches of seticore or seticore2 and detection SNR or timing comparisons
can be made between branches over multiple runs of this notebook.  For each run of this notebook, 
the desired seticore branch needs to be checked out and compiled, and 
the string "test_case" needs to be set in 01_seti_end_to_end.ipynb to indicate the branch.

In [1]:
%load_ext autoreload
%autoreload 2

raw_files_are_undefined = False

raw_backup_subdir = 'raw_multichirp/'
if (0):
    raw_file_stem_list = [
        'gbt-chirp80-0.0040V-6002.20-6003.80-m10.0-10.0-Hzsec-22.91sec']
elif (0):
    raw_file_stem_list = [
        'gbt-chirp80-0.0040V-6002.20-6003.80-m10.0-10.0-Hzsec-366.50sec',
        'meerkat-chirp80-0.0040V-1502.30-1502.70-m10.0-10.0-Hzsec-321.49sec',
        'vlass-chirp80-0.0040V-1401.80-1402.40-m10.0-10.0-Hzsec-8.00sec']
elif(1):
    raw_file_stem_list = [
        'gbt-chirp80-0.0040V-6002.20-6003.80-m10.0-10.0-Hzsec-22.91sec',
        'gbt-chirp80-0.0040V-6002.20-6003.80-m10.0-10.0-Hzsec-45.81sec',
        'gbt-chirp80-0.0040V-6002.20-6003.80-m10.0-10.0-Hzsec-91.63sec',
        'gbt-chirp80-0.0040V-6002.20-6003.80-m10.0-10.0-Hzsec-183.25sec']
        # 'gbt-chirp80-0.0040V-6002.20-6003.80-m10.0-10.0-Hzsec-366.50sec']
elif (0):
    raw_file_stem_list = [
        'meerkat-chirp80-0.0040V-1502.30-1502.70-m10.0-10.0-Hzsec-321.49sec',
        'meerkat-chirp80-0.0040V-1502.30-1502.70-m10.0-10.0-Hzsec-160.75sec',
        'meerkat-chirp80-0.0040V-1502.30-1502.70-m10.0-10.0-Hzsec-80.37sec']
elif (0):
    raw_file_stem_list = ['meerkat-chirp80-0.0040V-1502.30-1502.70-m10.0-10.0-Hzsec-321.49sec']
elif (0):
    raw_file_stem_list = ['vlass-chirp80-0.0040V-1401.80-1402.40-m10.0-10.0-Hzsec-8.00sec']
elif(0):
    raw_file_stem_list = [
        'gbt-chirp80-0.0040V-6002.20-6003.80-m10.0-10.0-Hzsec-183.25sec',
        'gbt-chirp80-0.0040V-6002.20-6003.80-m10.0-10.0-Hzsec-366.50sec']
elif (1):
    raw_file_stem_list = ['gbt-chirp80-0.0040V-6002.20-6003.80-m10.0-10.0-Hzsec-183.25sec']
elif (0):
    raw_file_stem_list = ['vlass-chirp80-0.0040V-1401.80-1402.40-m10.0-10.0-Hzsec-8.00sec']
elif (0):
    raw_backup_subdir = 'raw_voyager/'
    raw_file_stem_list = ['blc23_guppi_59046_81310_DIAG_VOYAGER-1_0015'] # 23.7 dB, -.504 Hz/sec
elif (0):
    raw_backup_subdir = 'raw_voyager/'
    raw_file_stem_list = ['blc3_guppi_57386_VOYAGER1_0004'] 
elif (0):
    raw_backup_subdir = 'raw_gbt_57388_HIP/'
    raw_file_stem_list = ['blc1_guppi_57388_HIP113357_0010',
                          'blc2_guppi_57388_HIP113357_0010',
                          'blc3_guppi_57388_HIP113357_0010',
                          'blc4_guppi_57388_HIP113357_0010',
                          'blc5_guppi_57388_HIP113357_0010',
                          'blc6_guppi_57388_HIP113357_0010'] 
elif (0):
    raw_backup_subdir = 'raw_gbt_57388_HIP/'
    raw_file_stem_list = ['blc1_guppi_57388_HIP113357_0012',
                          'blc2_guppi_57388_HIP113357_0012',
                          'blc3_guppi_57388_HIP113357_0012',
                          'blc4_guppi_57388_HIP113357_0012',
                          'blc5_guppi_57388_HIP113357_0012',
                          'blc6_guppi_57388_HIP113357_0012'] 
elif (0):
    raw_backup_subdir = 'raw_gbt_59103_HIP/'
    raw_file_stem_list = ['blc40_guppi_59103_03394_DIAG_HIP95631_0015',
                          'blc41_guppi_59103_03394_DIAG_HIP95631_0015',
                          'blc42_guppi_59103_03394_DIAG_HIP95631_0015',
                          'blc43_guppi_59103_03394_DIAG_HIP95631_0015',
                          'blc44_guppi_59103_03394_DIAG_HIP95631_0015',
                          'blc45_guppi_59103_03394_DIAG_HIP95631_0015',
                          'blc46_guppi_59103_03394_DIAG_HIP95631_0015',
                          'blc47_guppi_59103_03394_DIAG_HIP95631_0015'] 
elif (1):
    raw_backup_subdir = 'raw_gbt_59103_Kepler/'
    raw_file_stem_list = ['blc40_guppi_59103_01984_DIAG_KEPLER-160_0010',
                          'blc41_guppi_59103_01984_DIAG_KEPLER-160_0010',
                          'blc42_guppi_59103_01984_DIAG_KEPLER-160_0010',
                          'blc43_guppi_59103_01984_DIAG_KEPLER-160_0010',
                          'blc44_guppi_59103_01984_DIAG_KEPLER-160_0010',
                          'blc45_guppi_59103_01984_DIAG_KEPLER-160_0010',
                          'blc46_guppi_59103_01984_DIAG_KEPLER-160_0010',
                          'blc47_guppi_59103_01984_DIAG_KEPLER-160_0010'] 
elif (1):
    raw_backup_subdir = 'raw_gbt_59103_TIC/'
    raw_file_stem_list = ['blc40_guppi_59103_03721_DIAG_TIC159107668_0016',
                          'blc41_guppi_59103_03721_DIAG_TIC159107668_0016',
                          'blc42_guppi_59103_03721_DIAG_TIC159107668_0016',
                          'blc43_guppi_59103_03721_DIAG_TIC159107668_0016',
                          'blc44_guppi_59103_03721_DIAG_TIC159107668_0016',
                          'blc45_guppi_59103_03721_DIAG_TIC159107668_0016',
                          'blc46_guppi_59103_03721_DIAG_TIC159107668_0016',
                          'blc47_guppi_59103_03721_DIAG_TIC159107668_0016'] 

plot_spectra = False
plot_snr_vs_f = False
zoom_bw_MHz = 1.0
do_profile = False


#### Run end-to-end test over parameter sweeps

In [2]:
n_raw_file = len(raw_file_stem_list)


for i_raw in range(n_raw_file):

    raw_file_stem = raw_file_stem_list[i_raw]
    print(f'\n\nRaw file {i_raw+1} of {n_raw_file}: {raw_file_stem}\n\n')

    try:      
        %run ./02_seti_param_sweep.ipynb
    except:
        print(f'\n\nError Occurred for Raw file {raw_file_stem}, skip\n\n')

    if (1):
        # delete raw file to limit local file storage
        if (i_raw+1<n_raw_file):
            try:
                print('Deleting ',raw_file_delete_spec)
                os.system('rm '+raw_file_delete_spec)
            except:
                print(' ')

            



Raw file 1 of 4: gbt-chirp80-0.0040V-6002.20-6003.80-m10.0-10.0-Hzsec-22.91sec


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
sigma_drift = [0.75 1.5  3.  ]
raw_file_stem = 'gbt-chirp80-0.0040V-6002.20-6003.80-m10.0-10.0-Hzsec-22.91sec'
/home/kmh4005/data_files/raw_files/gbt-chirp80-0.0040V-6002.20-6003.80-m10.0-10.0-Hzsec-22.91sec.0000.raw
Raw file found in backup directory, copying
/mnt/d/data_files/raw_multichirp/gbt-chirp80-0.0040V-6002.20-6003.80-m10.0-10.0-Hzsec-22.91sec.0000.raw
8.1G -rw-rw-rw- 1 kmh4005 kmh4005 8.1G Nov 14 21:44 /home/kmh4005/data_files/raw_files/gbt-chirp80-0.0040V-6002.20-6003.80-m10.0-10.0-Hzsec-183.25sec.0000.raw
1.1G -rw-rw-rw- 1 kmh4005 kmh4005 1.1G Dec  5 10:17 /home/kmh4005/data_files/raw_files/gbt-chirp80-0.0040V-6002.20-6003.80-m10.0-10.0-Hzsec-22.91sec.0000.raw
257M -rw-rw-rw- 1 kmh4005 kmh4005 257M Nov  8 13:29 /home/kmh4005/data_files/raw_files/gbt-chirp80-0.0040V-6002.20-6003.80-m10.0-10.0-Hzsec-5.73sec

dedoppler elapsed time: 1s



Seticore Search complete, 1.50 sec 0.37 sec/coarse channel
[21.57 21.88 22.08]
E(SNR)    [21.57 21.88 22.08] dB
fig 09: plot_dets2=True, isguppi=False, 
fig_dir='./chirp_gbt_plots/', fig_name_base='gbt-chirp80-0.0040V-6002.20-6003.80-m10.0-10.0-Hzsec-22.91sec-FineFFT-512K-Nsti-02'
raw_file_name='/home/kmh4005/data_files/raw_files/gbt-chirp80-0.0040V-6002.20-6003.80-m10.0-10.0-Hzsec-22.91sec.0000.raw' found, n_blocks_incr=8, n_blocks=8
raw_file_name='/home/kmh4005/data_files/raw_files/gbt-chirp80-0.0040V-6002.20-6003.80-m10.0-10.0-Hzsec-22.91sec.0001.raw' not found, n_raw_files=1, Total blocks=8
raw_file_base_name = '/home/kmh4005/data_files/raw_files/gbt-chirp80-0.0040V-6002.20-6003.80-m10.0-10.0-Hzsec-22.91sec.0000.raw'
Raw file found
Raw File size =   1024 MB, n_raw_files = 1
expected_file_size = 1024.0 MB excl header

GBT     , Tue Sep 22 00:24:27 2020, mjd_day=59114, TMC1    , ra_deg = 70.441, dec_deg = 25.691
ctr_freq_MHz = 6004.395, obs_bw_MHz = 11.71875, f_min_MHz = 5998.535, f

<Figure size 1000x600 with 0 Axes>

test_case='sc2'
sc_app_name='seticore2', search_z_threshold=20
/home/kmh4005/data_files/raw_files/gbt-chirp80-0.0040V-6002.20-6003.80-m10.0-10.0-Hzsec-22.91sec.0000.raw
Raw file found in raw directory
gbt-chirp80-0.0040V-6002.20-6003.80-m10.0-10.0-Hzsec-22.91sec
['gbt', 'chirp80', '0.0040V', '6002.20', '6003.80', 'm10.0', '10.0', 'Hzsec', '22.91sec']
n_chirp = 80 f_start1_MHz = 6002.2 f_stop1_MHz = 6003.8 sig_min_drift = -10.0 sig_max_drift = 10.0
raw_file_name='/home/kmh4005/data_files/raw_files/gbt-chirp80-0.0040V-6002.20-6003.80-m10.0-10.0-Hzsec-22.91sec.0000.raw' found, n_blocks_incr=8, n_blocks=8
raw_file_name='/home/kmh4005/data_files/raw_files/gbt-chirp80-0.0040V-6002.20-6003.80-m10.0-10.0-Hzsec-22.91sec.0001.raw' not found, n_raw_files=1, Total blocks=8
raw_file_base_name = '/home/kmh4005/data_files/raw_files/gbt-chirp80-0.0040V-6002.20-6003.80-m10.0-10.0-Hzsec-22.91sec.0000.raw'
Raw file found
Raw File size =   1024 MB, n_raw_files = 1
expected_file_size = 1024.0 MB excl heade

...


Seticore Search complete, 1.71 sec 0.43 sec/coarse channel
[22.56 20.19 18.2 ]
E(SNR)    [22.56 20.19 18.2 ] dB
fig 09: plot_dets2=True, isguppi=False, 
fig_dir='./chirp_gbt_plots/', fig_name_base='gbt-chirp80-0.0040V-6002.20-6003.80-m10.0-10.0-Hzsec-183.25sec-FineFFT-2048K-Nsti-32'
raw_file_name='/home/kmh4005/data_files/raw_files/gbt-chirp80-0.0040V-6002.20-6003.80-m10.0-10.0-Hzsec-183.25sec.0000.raw' found, n_blocks_incr=64, n_blocks=64
raw_file_name='/home/kmh4005/data_files/raw_files/gbt-chirp80-0.0040V-6002.20-6003.80-m10.0-10.0-Hzsec-183.25sec.0001.raw' not found, n_raw_files=1, Total blocks=64
raw_file_base_name = '/home/kmh4005/data_files/raw_files/gbt-chirp80-0.0040V-6002.20-6003.80-m10.0-10.0-Hzsec-183.25sec.0000.raw'
Raw file found
Raw File size =   8192 MB, n_raw_files = 1
expected_file_size = 8192.0 MB excl header

GBT     , Tue Sep 22 00:24:27 2020, mjd_day=59114, TMC1    , ra_deg = 70.441, dec_deg = 25.691
ctr_freq_MHz = 6004.395, obs_bw_MHz = 11.71875, f_min_MHz = 599

<Figure size 1000x600 with 0 Axes>

test_case='sc2'
sc_app_name='seticore2', search_z_threshold=20
/home/kmh4005/data_files/raw_files/gbt-chirp80-0.0040V-6002.20-6003.80-m10.0-10.0-Hzsec-183.25sec.0000.raw
Raw file found in raw directory
gbt-chirp80-0.0040V-6002.20-6003.80-m10.0-10.0-Hzsec-183.25sec
['gbt', 'chirp80', '0.0040V', '6002.20', '6003.80', 'm10.0', '10.0', 'Hzsec', '183.25sec']
n_chirp = 80 f_start1_MHz = 6002.2 f_stop1_MHz = 6003.8 sig_min_drift = -10.0 sig_max_drift = 10.0
raw_file_name='/home/kmh4005/data_files/raw_files/gbt-chirp80-0.0040V-6002.20-6003.80-m10.0-10.0-Hzsec-183.25sec.0000.raw' found, n_blocks_incr=64, n_blocks=64
raw_file_name='/home/kmh4005/data_files/raw_files/gbt-chirp80-0.0040V-6002.20-6003.80-m10.0-10.0-Hzsec-183.25sec.0001.raw' not found, n_raw_files=1, Total blocks=64
raw_file_base_name = '/home/kmh4005/data_files/raw_files/gbt-chirp80-0.0040V-6002.20-6003.80-m10.0-10.0-Hzsec-183.25sec.0000.raw'
Raw file found
Raw File size =   8192 MB, n_raw_files = 1
expected_file_size = 8192.0 MB e

dedoppler elapsed time: 1s



Seticore Search complete, 1.64 sec 0.41 sec/coarse channel
[19.72 17.42 15.97]
E(SNR)    [19.72 17.42 15.97] dB
fig 09: plot_dets2=True, isguppi=False, 
fig_dir='./chirp_gbt_plots/', fig_name_base='gbt-chirp80-0.0040V-6002.20-6003.80-m10.0-10.0-Hzsec-183.25sec-FineFFT-2048K-Nsti-64'
Writing results to file ./chirp_gbt_plots/gbt-chirp80-0.0040V-6002.20-6003.80-m10.0-10.0-Hzsec-183.25sec-sc2.npz
select_params =[[1048576, 32], [1048576, 4], [2097152, 2]]
fine_fft_size_list =array([ 524288, 1048576, 2097152])
n_sti_list =array([ 2,  4,  8, 16, 32, 64])
select_params =[[1048576, 32], [1048576, 4], [2097152, 2]]
fine_fft_size_list =array([ 524288, 1048576, 2097152])
n_sti_list =array([ 2,  4,  8, 16, 32, 64])
sigma_drift = [0.75 1.5  3.  ]
./chirp_gbt_plots/21-gbt-chirp80-0.0040V-6002.20-6003.80-m10.0-10.0-Hzsec-183.25sec-fctr-2GHz-seticore-snr-time-sc2.png
./chirp_gbt_plots/21-gbt-chirp80-0.0040V-6002.20-6003.80-m10.0-10.0-Hzsec-183.25sec-fctr-3GHz-seticore-snr-time-sc2.png
./chirp_gbt_plo

In [3]:
import os

print('All Runs Done')

# Beep 2 times in WSL
os.system("powershell.exe '[console]::beep(261.6,700)'")
os.system("powershell.exe '[console]::beep(261.6,700)'")


All Runs Done


0